In [1]:
import tensortrade.env.default as default
from tensortrade.oms.exchanges import Exchange
from tensortrade.feed import Stream
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.instruments import Instrument
from tensortrade.agents import DQNAgent
from tensortrade.env.default.actions import BSH, ManagedRiskOrders
from tensortrade.env.default.rewards import RiskAdjustedReturns
from tensortrade.env.default.renderers import PlotlyTradingChart
from all_indicators import get_all_stock_indicators
from sklearn.preprocessing import MinMaxScaler
import multiprocessing
import pandas as pd
import numpy as np

pd.options.mode.use_inf_as_na = True

B:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
TKO_data = get_all_stock_indicators('TKO.PA')
STM_data = get_all_stock_indicators('STM.PA')
THEP_data = get_all_stock_indicators('THEP.PA')
BIM_data = get_all_stock_indicators('BIM.PA')
ERF_data = get_all_stock_indicators('ERF.PA')
TRI_data = get_all_stock_indicators('TRI.PA')
VIRP_data = get_all_stock_indicators('VIRP.PA')
AI_data = get_all_stock_indicators('AI.PA')
SU_data = get_all_stock_indicators('SU.PA')
LTA_data = get_all_stock_indicators('LTA.PA')

NameError: name 'get_all_stock_indicators' is not defined

In [4]:
selected_var = pd.read_excel('feature_selection.xlsx', sheet_name='ALL')

def drop_corr_col(stock_data, ticker):
    relevant_cols = selected_var[ticker].dropna()
    relevant_cols = relevant_cols[relevant_cols!='Sortino 10']
    filtered_data = stock_data.copy().drop(columns=['Open', 'High', 'Low', 'Close', 'Volume','Returns','Log Returns'])
    filtered_data = filtered_data[relevant_cols]
    corr_matrix = filtered_data.corr()
    upperMatrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    corrFeatures = [column for column in upperMatrix.columns if any(upperMatrix[column] > 0.95)]
    print(corrFeatures)
    return pd.concat([filtered_data.drop(columns=corrFeatures),stock_data[['Open', 'High', 'Low', 'Close', 'Volume','Returns','Log Returns']]],axis=1)

In [5]:
selected_STM_data = drop_corr_col(STM_data, 'STM')
selected_THEP_data = drop_corr_col(THEP_data, 'THEP')
selected_BIM_data = drop_corr_col(BIM_data, 'BIM')
selected_TKO_data = drop_corr_col(TKO_data, 'TKO')
selected_ERF_data = drop_corr_col(ERF_data, 'ERF')
selected_TRI_data = drop_corr_col(TRI_data, 'TRI')
selected_VIRP_data = drop_corr_col(VIRP_data, 'VIRP')
selected_AI_data = drop_corr_col(AI_data, 'AI')
selected_SU_data = drop_corr_col(SU_data, 'SU')
selected_LTA_data = drop_corr_col(LTA_data, 'LTA')

['trend_vortex_ind_diff', 'Sharpe 14', 'Sharpe 18', 'Sharpe 16', 'Beta 24']
['Sharpe 12']
['Sharpe 18']
['Sharpe 14', 'Sharpe 16', 'Sharpe 18', 'Sortino 24', 'Sortino 26', 'Beta 24']
['Sharpe 126', 'Sortino 24', 'Sharpe 16']
[]
['Sortino 22']
['Sharpe 16', 'Sortino 18']
['Sortino 16', 'Sharpe 18']
['volatility_bbh', 'Sortino 126', 'trend_ichimoku_b', 'rsi10']


In [6]:
from sklearn.model_selection import train_test_split

def split_data(data):
    X = data.copy().drop(columns=['Log Returns','Returns'])
    y = data.copy()['Returns']

    X_train_test, X_valid, y_train_test, y_valid = \
        train_test_split(X, y, train_size=0.8, test_size=0.2, shuffle=False)

    X_train, X_test, y_train, y_test = \
        train_test_split(X_train_test, y_train_test, train_size=0.7, test_size=0.3, shuffle=False)

    return X_train, X_test, X_valid, y_train, y_test, y_valid

TKO_X_train, TKO_X_test, TKO_X_valid, TKO_y_train, TKO_y_test, TKO_y_valid = split_data(selected_TKO_data)
STM_X_train, STM_X_test, STM_X_valid, STM_y_train, STM_y_test, STM_y_valid = split_data(selected_STM_data)
THEP_X_train, THEP_X_test, THEP_X_valid, THEP_y_train, THEP_y_test, THEP_y_valid = split_data(selected_THEP_data)
BIM_X_train, BIM_X_test, BIM_X_valid, BIM_y_train, BIM_y_test, BIM_y_valid = split_data(selected_BIM_data)
ERF_X_train, ERF_X_test, ERF_X_valid, ERF_y_train, ERF_y_test, ERF_y_valid = split_data(selected_ERF_data)
TRI_X_train, TRI_X_test, TRI_X_valid, TRI_y_train, TRI_y_test, TRI_y_valid = split_data(selected_TRI_data)
VIRP_X_train, VIRP_X_test, VIRP_X_valid, VIRP_y_train, VIRP_y_test, VIRP_y_valid = split_data(selected_VIRP_data)
AI_X_train, AI_X_test, AI_X_valid, AI_y_train, AI_y_test, AI_y_valid = split_data(selected_AI_data)
SU_X_train, SU_X_test, SU_X_valid, SU_y_train, SU_y_test, SU_y_valid = split_data(selected_SU_data)
LTA_X_train, LTA_X_test, LTA_X_valid, LTA_y_train, LTA_y_test, LTA_y_valid = split_data(selected_LTA_data)

In [7]:
import os
import pickle
cwd = os.getcwd()

TKO_train_pickle = os.path.join(cwd, 'TKO_X_train.pickle')
STM_train_pickle = os.path.join(cwd, 'STM_X_train.pickle')
THEP_train_pickle = os.path.join(cwd, 'THEP_X_train.pickle')
BIM_train_pickle = os.path.join(cwd, 'BIM_X_train.pickle')
ERF_train_pickle = os.path.join(cwd, 'ERF_X_train.pickle')
TRI_train_pickle = os.path.join(cwd, 'TRI_X_train.pickle')
VIRP_train_pickle = os.path.join(cwd, 'VIRP_X_train.pickle')
AI_train_pickle = os.path.join(cwd, 'AI_X_train.pickle')
SU_train_pickle = os.path.join(cwd, 'SU_X_train.pickle')
LTA_train_pickle = os.path.join(cwd, 'LTA_X_train.pickle')
TKO_test_pickle = os.path.join(cwd, 'TKO_X_test.pickle')
STM_test_pickle = os.path.join(cwd, 'STM_X_test.pickle')
THEP_test_pickle = os.path.join(cwd, 'THEP_X_test.pickle')
BIM_test_pickle = os.path.join(cwd, 'BIM_X_test.pickle')
ERF_test_pickle = os.path.join(cwd, 'ERF_X_test.pickle')
TRI_test_pickle = os.path.join(cwd, 'TRI_X_test.pickle')
VIRP_test_pickle = os.path.join(cwd, 'VIRP_X_test.pickle')
AI_test_pickle = os.path.join(cwd, 'AI_X_test.pickle')
SU_test_pickle = os.path.join(cwd, 'SU_X_test.pickle')
LTA_test_pickle = os.path.join(cwd, 'LTA_X_test.pickle')

TKO_X_train.to_pickle(TKO_train_pickle)
STM_X_train.to_pickle(STM_train_pickle)
THEP_X_train.to_pickle(THEP_train_pickle)
BIM_X_train.to_pickle(BIM_train_pickle)
ERF_X_train.to_pickle(ERF_train_pickle)
TRI_X_train.to_pickle(TRI_train_pickle)
VIRP_X_train.to_pickle(VIRP_train_pickle)
AI_X_train.to_pickle(AI_train_pickle)
SU_X_train.to_pickle(SU_train_pickle)
LTA_X_train.to_pickle(LTA_train_pickle)
TKO_X_test.to_pickle(TKO_test_pickle)
STM_X_test.to_pickle(STM_test_pickle)
THEP_X_test.to_pickle(THEP_test_pickle)
BIM_X_test.to_pickle(BIM_test_pickle)
ERF_X_test.to_pickle(ERF_test_pickle)
TRI_X_test.to_pickle(TRI_test_pickle)
VIRP_X_test.to_pickle(VIRP_test_pickle)
AI_X_test.to_pickle(AI_test_pickle)
SU_X_test.to_pickle(SU_test_pickle)
LTA_X_test.to_pickle(LTA_test_pickle)

In [2]:
import os
import pickle
cwd = os.getcwd()

TKO_train_pickle = os.path.join(cwd, 'TKO_X_train.pickle')
STM_train_pickle = os.path.join(cwd, 'STM_X_train.pickle')
THEP_train_pickle = os.path.join(cwd, 'THEP_X_train.pickle')
BIM_train_pickle = os.path.join(cwd, 'BIM_X_train.pickle')
ERF_train_pickle = os.path.join(cwd, 'ERF_X_train.pickle')
TRI_train_pickle = os.path.join(cwd, 'TRI_X_train.pickle')
VIRP_train_pickle = os.path.join(cwd, 'VIRP_X_train.pickle')
AI_train_pickle = os.path.join(cwd, 'AI_X_train.pickle')
SU_train_pickle = os.path.join(cwd, 'SU_X_train.pickle')
LTA_train_pickle = os.path.join(cwd, 'LTA_X_train.pickle')
TKO_test_pickle = os.path.join(cwd, 'TKO_X_test.pickle')
STM_test_pickle = os.path.join(cwd, 'STM_X_test.pickle')
THEP_test_pickle = os.path.join(cwd, 'THEP_X_test.pickle')
BIM_test_pickle = os.path.join(cwd, 'BIM_X_test.pickle')
ERF_test_pickle = os.path.join(cwd, 'ERF_X_test.pickle')
TRI_test_pickle = os.path.join(cwd, 'TRI_X_test.pickle')
VIRP_test_pickle = os.path.join(cwd, 'VIRP_X_test.pickle')
AI_test_pickle = os.path.join(cwd, 'AI_X_test.pickle')
SU_test_pickle = os.path.join(cwd, 'SU_X_test.pickle')
LTA_test_pickle = os.path.join(cwd, 'LTA_X_test.pickle')

In [3]:
def separate_render_features(stock_data, ticker):
    #ohlc prices for render
    stock_ohlc = stock_data[['Open','High','Low','Close','Volume']].copy()
    stock_ohlc['date'] = stock_ohlc.index + pd.DateOffset(hours=2)
    stock_ohlc = stock_ohlc.add_prefix(f"{ticker}:")
    
    #all features to train from + minmax scaling
    scaler = MinMaxScaler()
    stock_features = stock_data.copy()
    stock_features = stock_features.add_prefix(f"{ticker}:")
    scaler.fit(stock_features)
    stock_features_scaled = pd.DataFrame(scaler.fit_transform(stock_features), columns = stock_features.columns, index = stock_features.index)
    return stock_ohlc, stock_features_scaled

def get_price_stream(stock_renders, tickers): #list
    stock_price_stream_list = []
    for i in range(len(stock_renders)):
        stock_price_stream_list.append(Stream.source(list(stock_renders[i][f"{tickers[i]}:Close"]), dtype="float").rename(f"EUR-{tickers[i].replace('.PA', '')}"))
    return stock_price_stream_list

In [4]:
def create_env(config):
    TKO_data = pd.read_pickle(config['TKO_filename'])
    STM_data = pd.read_pickle(config['STM_filename'])
    THEP_data = pd.read_pickle(config['THEP_filename'])
    BIM_data = pd.read_pickle(config['BIM_filename'])
    ERF_data = pd.read_pickle(config['ERF_filename'])
    TRI_data = pd.read_pickle(config['TRI_filename'])
    VIRP_data = pd.read_pickle(config['VIRP_filename'])
    AI_data = pd.read_pickle(config['AI_filename'])
    SU_data = pd.read_pickle(config['SU_filename'])
    LTA_data = pd.read_pickle(config['LTA_filename'])
    
    TKO_render, TKO_scaled_features = separate_render_features(TKO_data, 'TKO.PA')
    STM_render, STM_scaled_features = separate_render_features(STM_data, 'STM.PA')
    THEP_render, THEP_scaled_features = separate_render_features(THEP_data, 'THEP.PA')
    BIM_render, BIM_scaled_features = separate_render_features(BIM_data, 'BIM.PA')   
    ERF_render, ERF_scaled_features = separate_render_features(ERF_data, 'ERF.PA')
    TRI_render, TRI_scaled_features = separate_render_features(TRI_data, 'TRI.PA')
    VIRP_render, VIRP_scaled_features = separate_render_features(VIRP_data, 'VIRP.PA')
    AI_render, AI_scaled_features = separate_render_features(AI_data, 'AI.PA')       
    SU_render, SU_scaled_features = separate_render_features(SU_data, 'SU.PA')
    LTA_render, LTA_scaled_features = separate_render_features(LTA_data, 'LTA.PA')     
    
    price_streams = get_price_stream([TKO_render, STM_render, THEP_render, BIM_render, ERF_render, TRI_render, VIRP_render, AI_render, SU_render, LTA_render], ["TKO.PA", "STM.PA", "THEP.PA", "BIM.PA", "ERF.PA", "TRI.PA","VIRP.PA", "AI.PA", "SU.PA", "LTA.PA"])
    euronext = Exchange('euronext', service=execute_order)(price_streams[0], price_streams[1], price_streams[2], price_streams[3], price_streams[4], price_streams[5], price_streams[6], price_streams[7], price_streams[8], price_streams[9])

    all_scaled_features = pd.concat([TKO_scaled_features, STM_scaled_features, THEP_scaled_features, BIM_scaled_features, ERF_scaled_features, TRI_scaled_features, VIRP_scaled_features, AI_scaled_features, SU_scaled_features, LTA_scaled_features], axis=1)
    with NameSpace("euronext"):
        features = [Stream.source(list(all_scaled_features[feature]), dtype="float").rename(feature) for feature in all_scaled_features.columns]
    all_features_feed = DataFeed(features)
    all_features_feed.compile()

    EUR = Instrument('EUR', 4, 'Euro')
    TKO = Instrument('TKO', 4, 'Tikehau')
    STM = Instrument('STM', 4, 'STMicroelectronics')
    THEP = Instrument('THEP', 4, 'Thermador')
    BIM = Instrument('BIM', 4, 'Biomerieux')
    ERF = Instrument('ERF', 4, 'Eurofins')
    TRI = Instrument('TRI', 4, 'Trigano')
    VIRP = Instrument('VIRP', 4, 'Virbac')
    AI = Instrument('AI', 4, 'Air Liquide')
    SU = Instrument('SU', 4, 'Schneider Electric')
    LTA = Instrument('LTA', 4, 'Altamir')
    
    cash = Wallet(euronext, 10000 * EUR)
    asset_tko = Wallet(euronext, 0 * TKO)
    asset_stm = Wallet(euronext, 0 * STM)
    asset_thep = Wallet(euronext, 0 * THEP)
    asset_bim = Wallet(euronext, 0 * BIM)
    asset_erf = Wallet(euronext, 0 * ERF)
    asset_tri = Wallet(euronext, 0 * TRI)
    asset_virp = Wallet(euronext, 0 * VIRP)
    asset_ai = Wallet(euronext, 0 * AI)
    asset_su = Wallet(euronext, 0 * SU)
    asset_lta = Wallet(euronext, 0 * LTA)
    
    portfolio = Portfolio(EUR, [cash, asset_tko, asset_stm, asset_thep, asset_bim, asset_erf, asset_tri, asset_virp, asset_ai, asset_su, asset_lta])

    reward_scheme = RiskAdjustedReturns(return_algorithm='sortino', window_size=75)
    action_scheme = ManagedRiskOrders()

    chart_renderer = PlotlyTradingChart(
        display=True,  # show the chart on screen (default)
        height=800,  # affects both displayed and saved file height. None for 100% height.
        save_format="html",  # save the chart to an HTML file
        auto_open_html=True,  # open the saved HTML chart in a new browser tab
    )
    
    renderer_feed = DataFeed([
        Stream.source(list(TKO_render["TKO.PA:date"])).rename("date"),
        Stream.source(list(TKO_render["TKO.PA:Open"]), dtype="float").rename("open"),
        Stream.source(list(TKO_render["TKO.PA:High"]), dtype="float").rename("high"),
        Stream.source(list(TKO_render["TKO.PA:Low"]), dtype="float").rename("low"),
        Stream.source(list(TKO_render["TKO.PA:Close"]), dtype="float").rename("close"), 
        Stream.source(list(TKO_render["TKO.PA:Volume"]), dtype="float").rename("volume"),
    ])

    env = default.create(
        portfolio=portfolio,
        action_scheme=action_scheme,
        reward_scheme=reward_scheme,
        feed=all_features_feed,
        renderer_feed=renderer_feed,
        renderer=chart_renderer,
        window_size=config["window_size"],
        max_allowed_loss=config["max_allowed_loss"]
    )
    
    return env


In [5]:
import ray
from ray import tune
from ray.tune.registry import register_env

ray.init(num_cpus=4,
         include_dashboard=True,
         ignore_reinit_error=True)

register_env("TradingEnv", create_env)

2022-12-15 22:05:09,704	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [ ]:
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.optuna import OptunaSearch

LR = tune.loguniform(1e-5, 1e-2)
GAMMA = tune.uniform(0.8, 0.9999)
LAMBDA = tune.uniform(0.1, 0.8)
VF_LOSS_COEFF = tune.uniform(0.01, 1.0)
ENTROPY_COEFF = tune.uniform(1e-8, 1e-1)

checkpoint_metric = 'episode_reward_mean'

# Specific configuration keys that will be used during training
env_config_training = {
    "TKO_filename": TKO_train_pickle, 
    "STM_filename": STM_train_pickle,
    "THEP_filename": THEP_train_pickle, 
    "BIM_filename": BIM_train_pickle,
    "ERF_filename": ERF_train_pickle, 
    "TRI_filename": TRI_train_pickle,
    "VIRP_filename": VIRP_train_pickle, 
    "AI_filename": AI_train_pickle,
    "SU_filename": SU_train_pickle, 
    "LTA_filename": LTA_train_pickle,
    "max_allowed_loss": 0.10, 
    "window_size": 30 
}
# Specific configuration keys that will be used during evaluation (only the overridden ones)
env_config_evaluation = {
    "TKO_filename": TKO_test_pickle, 
    "STM_filename": STM_test_pickle,
    "THEP_filename": THEP_test_pickle, 
    "BIM_filename": BIM_test_pickle,
    "ERF_filename": ERF_test_pickle, 
    "TRI_filename": TRI_test_pickle,
    "VIRP_filename": VIRP_test_pickle, 
    "AI_filename": AI_test_pickle,
    "SU_filename": SU_test_pickle, 
    "LTA_filename": LTA_test_pickle,
    "max_allowed_loss": 1, 
    "window_size": 30 
}
# Spec

search_alg = OptunaSearch()
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=4)

scheduler = ASHAScheduler()

import time
start = time.time()
analysis = tune.run(
    "PPO",
    stop={
        "episode_reward_mean": 2000,
        "training_iteration": 35,
    },
    verbose=3,
    config={
        "env": "TradingEnv",
        "env_config": env_config_training,
        "log_level": "ERROR",
        #"log_level": "INFO",
        #"log_level": "DEBUG",
        "framework": "torch",
        "ignore_worker_failures": True,
        "num_workers": 2,
        "num_gpus": 0,
        "clip_rewards": True,
        "lr": LR,
        "lr_schedule": [
            [0, 1e-1],
            [int(1e2), 1e-2],
            [int(1e3), 1e-3],
            [int(1e4), 1e-4],
            [int(1e5), 1e-5],
            [int(1e6), 1e-6],
            [int(1e7), 1e-7]
        ],
        "model": {
            "use_lstm": True,
            "lstm_cell_size": 512
        },
        "gamma": GAMMA,
        "observation_filter": "MeanStdFilter",
        "lambda": LAMBDA,
        "vf_share_layers": True,
        "vf_loss_coeff": VF_LOSS_COEFF,
        "entropy_coeff": ENTROPY_COEFF,
        "evaluation_interval": 1,  # Run evaluation on every iteration
        "evaluation_config": {
            "env_config": env_config_evaluation,  # The dictionary we built before (only the overriding keys to use in evaluation)
            "explore": False,  # We don't want to explore during evaluation. All actions have to be repeatable.
        },
    },
    metric=checkpoint_metric,
    mode="max",
    search_alg=search_alg,
    scheduler=scheduler,
    num_samples=10,  # Samples per hyperparameter combination. More averages out randomness. Less runs faster
    #resources_per_trial={"cpu": 2},
    keep_checkpoints_num=10,  # Keep the last 10 checkpoints
    checkpoint_freq=1,  # Do a checkpoint on each iteration (slower but you can pick more finely the checkpoint to use later)
    resume="AUTO",
)
taken = time.time() - start
print(f"Time taken: {taken:.2f} seconds.")
print(f"Best config: {analysis.best_config}")

C:\Users\remik\AppData\Local\Temp\ipykernel_12952\1214262178.py:2: DeprecationWarning: The module `ray.tune.suggest` has been moved to `ray.tune.search` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest` with `ray.tune.search`.
  from ray.tune.suggest import ConcurrencyLimiter
C:\Users\remik\AppData\Local\Temp\ipykernel_12952\1214262178.py:3: DeprecationWarning: The module `ray.tune.suggest.optuna` has been moved to `ray.tune.search.optuna` and the old location will be deprecated soon. Please adjust your imports to point to the new location. Example: Do a global search and replace `ray.tune.suggest.optuna` with `ray.tune.search.optuna`.
  from ray.tune.suggest.optuna import OptunaSearch
[I 2022-12-15 22:05:12,837] A new study created in memory with name: optuna
2022-12-15 22:05:12,864	INFO trial_runner.py:683 -- No local checkpoint was found. Ray Tune will now start a new expe

(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) B:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
(pid=14040)   warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) Windows fatal exception: code 0xc0000139
(pid=14040) 
(pid=14040) WARNING:tensorflow:From B:\Anaconda\lib\site-packages\tensorflow\python\util\deprecation.py:561: calling function (from tensorflow.python.eager.def_function) with experimental_relax_shapes is deprecated and will be removed in a future version.
(pid=14040) Instructions for updating:
(pid=14040) experimental_relax_shapes is depr

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,evaluation,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_TradingEnv_884a7705,60000,"{'num_env_steps_sampled': 60000, 'num_env_steps_trained': 60000, 'num_agent_steps_sampled': 60000, 'num_agent_steps_trained': 60000}",{},2022-12-16_02-21-08,False,499.11,{},908515,-1.4482e+06,-1.18284e+07,6,118,"{'episode_reward_max': 2.2871541084603124, 'episode_reward_min': 2.2871541084603124, 'episode_reward_mean': 2.287154108460313, 'episode_len_mean': 247.0, 'episode_media': {}, 'episodes_this_iter': 10, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124, 2.2871541084603124], 'episode_lengths': [247, 247, 247, 247, 247, 247, 247, 247, 247, 247]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.7540129742665231, 'mean_inference_ms': 5.815279164947077, 'mean_action_processing_ms': 0.08066226710790836, 'mean_env_wait_ms': 5.322742924182278, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'num_agent_steps_sampled_this_iter': 2470, 'num_env_steps_sampled_this_iter': 2470, 'timesteps_this_iter': 2470, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}",7b4a25127b3943c0abd2fdacbbba4f73,DESKTOP-N80GTVU,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 2.261252175800262, 'cur_kl_coeff': 0.6750000000000002, 'cur_lr': 5.4e-05, 'total_loss': 1.7451124548871992, 'policy_loss': -0.1929711052985807, 'vf_loss': 3.55090096789983, 'vf_explained_var': 0.3938513953198669, 'kl': 0.015172046910756323, 'entropy': 6.545931939668553, 'entropy_coeff': 6.548005712148715e-05}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 13485.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 60000, 'num_env_steps_trained': 60000, 'num_agent_steps_sampled': 60000, 'num_agent_steps_trained': 60000}",15,127.0.0.1,60000,60000,60000,4000,60000,4000,0,2,0,0,4000,"{'cpu_util_percent': 13.247117437722418, 'ram_util_percent': 77.66163701067616}",14040,{},{},{},"{'mean_raw_obs_processing_ms': 0.8659172322685121, 'mean_inference_ms': 5.928027824295603, 'mean_action_processing_ms': 0.08317311804656224, 'mean_env_wait_ms': 5.859601344632639, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 908515.1695934841, 'episode_reward_min': -11828426.45343953, 'episode_reward_mean': -1448196.865156557, 'episode_len_mean': 499.11, 'episode_media': {}, 'episodes_this_iter': 6, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [544744.0979353177, 4.962960878978196, -3.6728501747230626, 908512.5543750048, 2.1451596254984207, -305867.68361139763, 2.340083068650213, -229301.2290854197, -6362842.39160595, -1.6350130045119897, -1.0629755377860717, -11828426.45343953, -829818.5043064441, -6362843.559804704, -2.3373139258891027, -3682768.640156774, -621160.9144779942, 1.9333363855068233, -1.2363862697255652, -229298.8216918868, 2.005998224835111, -1659634.2431659705, -2.3558148474197322, -829814.914902

In [29]:
from keras.models import load_model
model = load_model('agents/policy_network__48f884a__20221213_175108.hdf5')
test = DQNAgent(env1, policy_network = model)

In [33]:
loaded_ag = DQNAgent(env1).restore('agents/policy_network__48f884a__20221213_175108.hdf5')

In [36]:
test.__dict__.keys()

dict_keys(['env', 'n_actions', 'observation_shape', 'policy_network', 'target_network', '_id'])

In [44]:
test.test()

AttributeError: 'DQNAgent' object has no attribute 'test'

In [ ]:
def print_quantstats_full_report(env, data, output='dqn_quantstats'):
    performance = pd.DataFrame.from_dict(env.action_scheme.portfolio.performance, orient='index')
    net_worth = performance['net_worth'].iloc[window_size:]
    returns = net_worth.pct_change().iloc[1:]

    # WARNING! The dates are fake and default parameters are used!
    returns.index = pd.date_range(start=data['date'].iloc[0], freq='1d', periods=returns.size)

    qs.reports.full(returns)
    qs.reports.html(returns, output=output + '.html')

print_quantstats_full_report(env, data)